***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/loan_approval_dataset.csv')

df['loan_status'] = df['loan_status'].map({' Approved': 1, ' Rejected': 0})
df_label = df['loan_status'].value_counts()

encoder = OrdinalEncoder()
df['education'] = encoder.fit_transform(df[['education']])
df['self_employed'] = encoder.fit_transform(df[['self_employed']])

df = df.drop(columns=['loan_id'])
df = df.dropna()

X = df.loc[:, df.columns != 'loan_status']
y = df['loan_status']

rus = RandomOverSampler(random_state=42)
X, y = rus.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

params = {
    'C': [1, 5, 10, 25, 50, 100]
}

model = LinearSVC(random_state=42)
cv = StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='f1_macro')

grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
rf_best = grid_search.best_estimator_
print("Best Estimator:", rf_best)

Best Parameters: {'C': 1}
Best Score: 0.4201361419582181
Best Estimator: LinearSVC(C=1, random_state=42)


In [ ]:
best = LinearSVC(C=1, random_state=42)
best.fit(X, y)

cv = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(best, X, y, cv=cv, scoring='f1_macro')
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

train_predictions = best.predict(X)
train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Cross-Validation Scores: [0.37540084 0.46819969 0.42041734 0.33291457 0.44688587 0.33375157
 0.5024278  0.33375157 0.53604778 0.3568314 ]
Mean CV Accuracy: 0.41066284270951436
Training Set Accuracy: 0.5141189759036144


##**Extracting weights and intercept values**

In [ ]:
b = best.intercept_
w = best.coef_
w = w.flatten()

[-2.54537936e-08 -1.09582228e-08  1.34862103e-08  1.01339342e-08
  2.31414917e-07 -4.57071222e-08  2.26204182e-05 -1.50802382e-07
  4.20958478e-08 -6.46203121e-08 -1.28535110e-07]
[4.15312103e-09]


##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['loan_status'])
predicted_label = best.predict(og_x)
results_df = pd.DataFrame({'Loan_Approval_Predicted': predicted_label})
index_class = results_df.index[results_df['Loan_Approval_Predicted'] == 0]

og_x_reset = og_x.reset_index(drop=True)
filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.5 MB/s eta 0:00:00
  Created wheel for z3: filename=z3-0.2.0-py3-none-any.whl size=26629 sha256=cc35781b585e87173aeda00795eea0ee59a839f3046858b9b33ddbf5a800c1a2
  Stored in directory: /root/.cache/pip/wheels/5a/b2/60/55b07a5084cad7ab411e395fb1440a2b1a19598bff535a3955
Successfully built z3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 3.1 MB/s eta 0:00:00


##**Optimizer -- step 1**

In [ ]:
import csv
from z3 import *
import time

def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    #dependents
    original_value = og_x_np[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    #education
    original_value1 = og_x_np[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    #self-employed
    original_value2 = og_x_np[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    #income amount
    opt.add(And(X[3] >= 35000, X[3] <= 40000))

    #loan amount
    opt.add(X[4] == og_x_np[4])

    #years
    opt.add(And(X[5] >= 5, X[5] <= 40))

    #credit score
    original_value6 = og_x_np[6]
    if original_value6 is not None:
      opt.add((X[6] == original_value6))
    opt.add(And(X[6] >= 600, X[6] <= 750))

    # Residential assets
    original_value7 = og_x_np[7]
    if original_value7 is not None:
      opt.add(X[7] == original_value7)

    # Commercial assets
    opt.add(And(X[8] >= 100, X[8] <= 250))

    # Luxury assets
    opt.add(And(X[9] >= 1000, X[9] <= 2000))

    # Bank assets
    original_value10 = og_x_np[10]
    if original_value10 is not None:
      opt.add(X[10] == original_value10)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])

    # Create the inequality constraint W * X + B >= 0
    inequality = dot_product + B > 0
    opt.add(inequality)

    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step1.csv"

for i in range(len(filtered_records)):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time()
    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step1.csv', names=column_names)

# Define a function to remove question marks from a single cell
def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]

optimized.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step1_optimized.csv', index=False)
left.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step1_failed.csv', index=False)

for i in range(len(optimized['Optimal_x'])):
  x = optimized['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 2**

In [ ]:
def step2(W, B, row):
    n = W.shape[0]
    opt = Optimize()
    X = [Real(f'x_{i}') for i in range(n)]

    #dependents
    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    #education
    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    #self-employed
    original_value2 = row[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    #income amount
    slack_income = Real('slack_income')
    slack_income_constraint = (slack_income > 0)
    opt.add(And(X[3] > 0, X[3] >= 35000 - slack_income, X[3] <= 40000 + slack_income))
    opt.add(slack_income_constraint)

    #loan amount
    original_value4 = row[4]
    if original_value4 is not None:
      opt.add(X[4] == original_value4)

    #years
    opt.add(And(X[5] >= 5, X[5] <= 40))

    #credit score
    slack_credit = Real('slack_credit')
    slack_credit_constraint = (slack_credit > 0)
    opt.add(And(X[6] >= 350, X[6] <= 900, X[6] >= 600 - slack_credit, X[6] <= 750 + slack_credit))
    opt.add(slack_credit_constraint)

    # Residential assets
    original_value7 = og_x_np[7]
    if original_value7 is not None:
      opt.add(X[7] == original_value7)

    # Commercial assets
    slack_ca = Real('slack_ca')
    slack_ca_constraint = (slack_ca > 0)
    opt.add(And(X[8] >= 0, X[8] >= 100 - slack_ca, X[8] <= 250 + slack_ca))
    opt.add(slack_ca_constraint)

    # Luxury assets
    slack_la = Real('slack_la')
    slack_la_constraint = (slack_la > 0)
    opt.add(And(X[9] >= 0, X[9] >= 1000 - slack_la, X[9] <= 2000 + slack_la))
    opt.add(slack_la_constraint)

    # Bank assets
    original_value10 = og_x_np[10]
    if original_value10 is not None:
      opt.add(X[10] == original_value10)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])

    # Create the inequality constraint W * X + B >= 0
    inequality = dot_product + B > 0
    opt.add(inequality)

    l1_distance = Sum([abs(row[i] - X[i]) for i in range(n)])

    objective = (slack_income + slack_credit + slack_ca + slack_la)
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        slack_income = model.eval(slack_income).as_decimal(15)
        print(f"slack_income: {slack_income}")

        slack_credit = model.eval(slack_credit).as_decimal(15)
        print(f"slack_credit: {slack_credit}")

        slack_ca = model.eval(slack_ca).as_decimal(15)
        print(f"slack_ca: {slack_ca}")

        slack_la = model.eval(slack_la).as_decimal(15)
        print(f"slack_la: {slack_la}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized, slack_income, slack_credit, slack_ca, slack_la
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '', '', '', '', '']

W = w
B = b[0]
output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step2.csv"
input_file = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step1_failed.csv')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()
    original_f, x_prime_optimized, cost, cost_optimized, slack_income, slack_credit, slack_ca, slack_la = step2(W, B, row)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_income] + [slack_credit] + [slack_ca] + [slack_la]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_income', 'slack_credit', 'Slack_ca', 'Slack_la']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step2.csv', names = column_names)

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 3**

In [ ]:
def step3(W, B, row, slack_value1, slack_value2, slack_value3, slack_value4):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    #dependents
    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    #education
    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    #self-employed
    original_value2 = row[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    #income amount
    opt.add(And(X[3] > 0, X[3] >= 35000 - slack_value1, X[3] <= 40000 + slack_value1))

    #loan amount
    opt.add(X[4] == og_x_np[4])

    #years
    opt.add(And(X[5] >= 5, X[5] <= 40))

    #credit score
    opt.add(And(X[6] >= 350, X[6] <= 900, X[6] >= 600 - slack_value2, X[6] <= 750 + slack_value2))

    original_value7 = og_x_np[7]
    if original_value7 is not None:
      opt.add(X[7] == original_value7)

    # Commercial assets
    opt.add(And(X[8] >= 0, X[8] >= 100 - slack_value3, X[8] <= 250 + slack_value3))

    # Luxury assets
    opt.add(And(X[9] >= 0, X[9] >= 1000 - slack_value4, X[9] <= 2000 + slack_value4))

    original_value10 = og_x_np[10]
    if original_value10 is not None:
      opt.add(X[10] == original_value10)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])

    # Create the inequality constraint W * X + B >= 0
    inequality = dot_product + B > 0

    opt.add(inequality)

    # Calculate the L0 distance
    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])

    objective = l0
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l0/Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value1 = last['Slack_income'].iloc[i]
    slack_value2 = last['slack_credit'].iloc[i]
    slack_value3 = last['Slack_ca'].iloc[i]
    slack_value4 = last['Slack_la'].iloc[i]

    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value1, slack_value2, slack_value3, slack_value4)

    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l0/Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l0/Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

## **Average runtimes for l0 and l1**

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l0/Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()
print('L0 runtime', l0_runtime)

In [ ]:
d3_prior = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step2.csv')
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step1_failed.csv')
d1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step1_optimized.csv')
d = pd.concat([d1, d3_initial])

l1_runtime = d.loc[:, 'Runtime'].mean()
print("L1 runtime", l1_runtime)

## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

21.688500727802037


### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_loans_l1/Step2.csv')
number_of_slacks_per_record = 4

count_s0 = 0
count_s1 = 0
count_s2 = 0
count_s3 = 0

total_count = 0
total_sum = 0
for i in range(len(relaxed_df)):
    if relaxed_df['Slack_income'].iloc[i] != 0:
        count_s0 = 1
    elif relaxed_df['Slack_income'].iloc[i] == 0:
        count_s0 = 0
    if relaxed_df['slack_credit'].iloc[i] != 0:
        count_s1 = 1
    elif relaxed_df['slack_credit'].iloc[i] == 0:
        count_s1 = 0
    if relaxed_df['Slack_ca'].iloc[i] != 0:
        count_s2 = 1
    elif relaxed_df['Slack_ca'].iloc[i] == 0:
        count_s2 = 0
    if relaxed_df['Slack_la'].iloc[i] != 0:
        count_s3 = 1
    elif relaxed_df['Slack_la'].iloc[i] == 0:
        count_s3 = 0

    total_count = count_s0 + count_s1 + count_s2 + count_s3
    total_sum += (total_count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100

print(average_per_dataset)

100.0
